In [29]:
from urllib import parse
import pandas as pd
import numpy as np
import lxml

In [30]:
class Stock:
    
    Exchange='C:/Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf-8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        res=[]#정규화 시작
        for i in range(len(data)):
            res.append(data[i] / np.linalg.norm(data, ord=1))
        res=np.array(res)
        return res

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 
        data=[]#정규화 시작
        ROE=np.array([])
        for i in range(7):
            ROE = sit[17+i][2:8].astype('float32')
            data.append(ROE/np.linalg.norm(ROE, ord=1))
        data=np.array(data)

        return data

    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]

    def toCsv(self,path,fileName): # 주식 종목코드 입력시 주식정보CSV로 저장해줌
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')

        sit = pd.DataFrame(tables[11])
        sit.to_csv(path+'/'+fileName+".csv",encoding='cp949')
        if(path[-1] == '/'):
            sit.to_csv(path+fileName+".csv",encoding='cp949')
        return

In [31]:
SS = Stock('005930')
#SK = Stock('000660')
#DW=Stock('049770')
#SFA=Stock('056190')
#GW=Stock('001570')

print('정규화 환율\n',Stock.exchangeRate(),'\n') #환율 나타내기
print('정규화 지표값\n',SS.idxNorm(),'\n') #지표값 정규화
SS.toCsv("C:/AI_20184218/","Samsung")#삼성재무재표 CSV파일화
SS.showChart() #삼성재무재표 바로보기

환율
 [0.01882938 0.0186708  0.0192739  0.02111267 0.02132189 0.01985938
 0.01929344 0.01906452 0.01839449 0.01870204 0.01797856 0.01777051
 0.0178373  0.01759046 0.01751272 0.01739071 0.01753523 0.01715764
 0.01704925 0.01673934 0.01638544 0.0165421  0.01669032 0.01664176
 0.01654921 0.01605805 0.01642601 0.01682833 0.0172453  0.01759016
 0.01767382 0.01775808 0.01823502 0.01796034 0.01803971 0.0177717
 0.01769426 0.01708864 0.01748296 0.01731327 0.01774046 0.01790941
 0.01758098 0.01710804 0.01758809 0.01722457 0.01682255 0.01665227
 0.01645681 0.01648495 0.01657586 0.0168954  0.01643579 0.01651693
 0.0164694  0.01649398 0.01598579] 

지표값
 [[0.27337417 0.1241352  0.14291361 0.19608617 0.16445936 0.09903143]
 [0.27757356 0.12287896 0.14126132 0.1968326  0.16430996 0.09714367]
 [0.21853007 0.11485163 0.13933831 0.20956977 0.19415222 0.12355801]
 [0.13861348 0.1471871  0.15455273 0.17104499 0.19023564 0.19836608]
 [0.13715614 0.13715614 0.29000387 0.13986827 0.14732662 0.14848895]
 [0.075

,IFRS(연결),Annual,Annual.1,Annual.2,Annual.3,Annual.4,Annual.5,Annual.6,Annual.7
0,IFRS(연결),2017/12,2018/12,2019/12,2020/12,2021/12,2022/12(E),2023/12(E),2024/12(E)
1,매출액,2395754,2437714,2304009,2368070,2796048,3050402,2926868,3147532
2,영업이익,536450,588867,277685,359939,516339,461365,287742,444324
3,영업이익(발표기준),536450,588867,277685,359939,516339,NaN,NaN,NaN
4,당기순이익,421867,443449,217389,264078,399075,369550,235368,354273
5,지배주주순이익,413446,438909,215051,260908,392438,363532,231334,348443
6,비지배주주순이익,8422,4540,2338,3170,6637,NaN,NaN,NaN
7,자산총계,3017521,3393572,3525645,3782357,4266212,4612083,4785157,5148194
8,부채총계,872607,916041,896841,1022877,1217212,1225914,1256209,1356846
9,자본총계,2144914,2477532,2628804,2759480,3048999,3386169,3528948,3791348
